In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

In [3]:
"""Group 15 data science project
DATASET:
(https://www.kaggle.com/competitions/playground-series-s4e11/data)

PROCESS:
1. Load and clean the data
2. EDA - Exploring data
3. Data Pre-processing
4. Creating model
5. Cross-validated training
6. Evaluation of model
"""

'Group 15 data science project\nDATASET:\n(https://www.kaggle.com/competitions/playground-series-s4e11/data)\n\nPROCESS:\n1. Load and clean the data\n2. EDA - Exploring data\n3. Data Pre-processing\n4. Creating model\n5. Cross-validated training\n6. Evaluation of model\n'

In [5]:
"""
1. Loading and cleaning data
"""

# Load the data
train_df = pd.read_csv('train.csv')

# Display the first 5 rows of the data
# train_df.head()

# Investigating Null values
print(train_df.isnull().sum())

train_df.fillna(0, inplace=True)

print(train_df.isnull().sum())

id                                            0
Name                                          0
Gender                                        0
Age                                           0
City                                          0
Working Professional or Student               0
Profession                                36630
Academic Pressure                        112803
Work Pressure                             27918
CGPA                                     112802
Study Satisfaction                       112803
Job Satisfaction                          27910
Sleep Duration                                0
Dietary Habits                                4
Degree                                        2
Have you ever had suicidal thoughts ?         0
Work/Study Hours                              0
Financial Stress                              4
Family History of Mental Illness              0
Depression                                    0
dtype: int64
id                         

In [6]:
"""
2. EDA -
"""

print('Unique values in column "Gender"', train_df['Gender'].unique())
print('Unique values in column "Working Professional or Student"', train_df['Working Professional or Student'].unique())
print('Unique values in column "Family History of Mental Illness"', train_df['Family History of Mental Illness'].unique())
print('Unique values in column "Have you ever had suicidal thoughts ?"', train_df['Have you ever had suicidal thoughts ?'].unique())
# print('Unique values in column "Dietry Habits"', train_df['Dietary Habits'].unique())


# print('Unique values in column "Profession"', train_df['Profession'].unique())
# print('Unique values in column "Degree"', train_df['Degree'].unique())

Unique values in column "Gender" ['Female' 'Male']
Unique values in column "Working Professional or Student" ['Working Professional' 'Student']
Unique values in column "Family History of Mental Illness" ['No' 'Yes']
Unique values in column "Have you ever had suicidal thoughts ?" ['No' 'Yes']


In [7]:
"""
3. Data Pre-Processing

COLUMNS -
id: Can be dropped
Name: Can be dropped
Gender: binary encoding
age: Temporarily okay but will need normalised
city: Can be dropped
Working professional or Student: Binary encoding
Profession: One-hot-encoding (TEMPORARILY DROPPED)
Academic Pressure: Temporarily okay but will need normalised
Work Pressure: Temporarily okay but will need normalised
CGPA: replace NaN with 0 - also will need normalised
Study Satisfaction: replace NaN with 0 - also will need normalised
Job Satisfaction: replace NaN with 0 - also will need normalised
Sleep Duration: Ordinal Encoding (TEMPORARILY DROPPED)
Dietary Habits: Ordinal Encoding (TEMPORARILY DROPPED)
Degree: One-hot-encoding (TEMPORARILY DROPPED)
Suicidal Thoughts: Binary Encoding
Work/Study Hours: Normalised
Financial stress: Normalised
Mental Illness: Binary Encoding
Depression: Target variable (no pre-processing needed)
"""
# THIS WILL NEED REMOVED
train_df = pd.read_csv('train.csv')
train_df.fillna(0, inplace=True)

# Removing target Column
target = train_df.pop('Depression')


# Column mappings - Binary encoding
gender_mapping = {
    'Male': 0,
    'Female': 1
}

work_mapping = {
    'Working Professional': 0,
    'Student': 1,
}

boolean_mapping = {
    'Yes': 0,
    'No': 1
}

train_df['Have you ever had suicidal thoughts ?'] = train_df['Have you ever had suicidal thoughts ?'].map(boolean_mapping)

train_df['Family History of Mental Illness'] = train_df['Family History of Mental Illness'].map(boolean_mapping)

train_df['Gender'] = train_df['Gender'].map(gender_mapping)

train_df['Working Professional or Student'] = train_df['Working Professional or Student'].map(work_mapping)


# Dropping Unneeded Columns
columns_to_drop = ['id', 'Name', 'City']
train_df.drop(columns=columns_to_drop, inplace=True)

# TEMPORARILY DROPPING COLUMNS FOR FIRST DRAFT
columns_to_drop = ['Profession', 'Sleep Duration', 'Degree', 'Dietary Habits']
train_df.drop(columns=columns_to_drop, inplace=True)

train_df.head()

,Gender,Age,Working Professional or Student,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness
0,1,49.0,0,0.0,5.0,0.00,0.0,2.0,1,1.0,2.0,1
1,0,26.0,0,0.0,4.0,0.00,0.0,3.0,0,7.0,3.0,1
2,0,33.0,1,5.0,0.0,8.97,2.0,0.0,0,3.0,1.0,1
3,0,22.0,0,0.0,5.0,0.00,0.0,1.0,0,10.0,1.0,0
4,1,30.0,0,0.0,1.0,0.00,0.0,1.0,0,9.0,4.0,0


In [8]:
"""
4. Creating Model / Training / Evaluating
"""

# Define model parameters
logistic_regression_params = {'max_iter': 1000, 'random_state': 42}
decision_tree_params = {'criterion': 'gini', 'max_depth': 3, 'random_state': 42}
random_forest_params = {'n_estimators': 100, 'max_depth': 5, 'random_state': 42}
svm_params = {'kernel': 'linear', 'C': 1.0}
knn_params = {'n_neighbors': 5}


# Define models
models = []
models.append({'Logistic_regression': LogisticRegression(**logistic_regression_params)})
models.append({'decision_tree': DecisionTreeClassifier(**decision_tree_params)})
models.append({'Random Forest': RandomForestClassifier(**random_forest_params)}),
models.append({'SVM': SVC(**svm_params)}),
models.append({'K-Nearest Neighbors': KNeighborsClassifier(**knn_params)})
# models.append({'name': Classificier})


model_results = []
for model_name, model in models.items():

  # Define stratified k-fold cross-validation
  n_splits = 5
  skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

  # For tracking scores
  fold = 1
  accuracies = []

  # Cross-validation loop
  for train_index, val_index in skf.split(train_df, target):
      # Split data
      X_train, X_val = train_df.iloc[train_index], train_df.iloc[val_index]
      y_train, y_val = target.iloc[train_index], target.iloc[val_index]

      # Train the model
      model.fit(X_train, y_train)

      # Predict on validation set
      y_pred = model.predict(X_val)

      # Evaluate accuracy
      acc = accuracy_score(y_val, y_pred)
      accuracies.append(acc)

      print(f"Fold {fold} Accuracy: {acc:.4f}")
      fold += 1

  model_results.append({model_name: np.mean(accuracies)})

# Average accuracy
# print(f"\nMean Accuracy across {n_splits} folds: {np.mean(accuracies):.4f}")
print(model_results)

NameError: name 'KNeighborsClassifier' is not defined